## Spoznavanje z okoljem

In [47]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

from os.path import join

Preberimo model osnovnega metabolizma E. coli. Model smo prenesli s strani http://bigg.ucsd.edu/models/e_coli_core.

In [48]:
model = cobra.io.read_sbml_model(join('models','e_coli_core.xml'))

In [56]:
r_biomass = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
r_biomass

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x245cd7b2e10
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [62]:
model.objective = r_biomass

In [65]:
model.optimize()

,fluxes,reduced_costs
PFK,7.477382,1.387779e-17
PFL,0.000000,-1.527746e-02
PGI,4.860861,0.000000e+00
PGK,-16.023526,3.824365e-18
PGL,4.959985,1.301043e-18
...,...,...
NADH16,38.534610,0.000000e+00
NADTRHD,0.000000,-2.546243e-03
NH4t,4.765319,0.000000e+00
O2t,21.799493,0.000000e+00


In [67]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


Rast celice je torej 0.874 $h^{-1}$

Rezultat optimizacije lahko primerjamo z orodjem [Escher-FBA](https://sbrg.github.io/escher-fba/#/app)

In [68]:
sol = model.optimize()
sol['EX_glc__D_e']

-10.0

Poskusimo spremeniti lastnosti medija. Kot vidimo zgoraj celica črpa veliko glukoze (pretok je 10 $mmol gDW^{-1} hr^{-1}$ (milimoli na gram suhe celične mase na uro - privzete enote v okolju COBRA). Kakšen je maksimalen dovod glukoze? Ali celica črpa vse kar ima na voljo?

In [75]:
r_ex_glc = model.reactions.get_by_id('EX_glc__D_e')
r_ex_glc

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x245dcd48c10
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-1.0
Upper bound,1000.0


Pretok je negativen, ampak reakcija je obrnjena! V orodju COBRA so vse *exchange* (*uptake* in *secretion*) reakcije take oblike. Kot vidimo, celica črpa vso glukozo, ki je na voljo. Poskusimo to zmanjšati.

In [76]:
r_ex_glc.bounds

(-1.0, 1000.0)

In [77]:
r_ex_glc.bounds = (0.0, 1000.0)
model.optimize()

C:\Users\miham\.conda\envs\metabolic\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


<Solution infeasible at 0x245dc0a5690>

Ups. Tole je bilo preveč. 

In [83]:
r_ex_glc.bounds = (-1.0, 1000.0)

In [84]:
model.optimize()

C:\Users\miham\.conda\envs\metabolic\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


<Solution infeasible at 0x245dc3e06d0>

Poskusimo analizo ponoviti v anaerobnih pogojih. Nastavili bomo uptake kisika.

In [80]:
r_ex_o2 = model.reactions.get_by_id('EX_o2_e')
r_ex_o2

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x245dcd2c350
Stoichiometry,o2_e <=> O2 O2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [85]:
r_ex_glc.bounds = (-10.0, 1000.0)
r_ex_o2.bounds = (0, 1000)

In [86]:
model.optimize()

,fluxes,reduced_costs
PFK,9.789459,-6.938894e-18
PFL,17.804674,0.000000e+00
PGI,9.956609,0.000000e+00
PGK,-19.437336,-1.912182e-18
PGL,0.000000,5.637851e-18
...,...,...
NADH16,0.000000,-5.538015e-03
NADTRHD,0.000000,-1.107603e-02
NH4t,1.154156,-2.856030e-17
O2t,0.000000,0.000000e+00


Produkcija biomase se je zmanjšala, prav tako pa se je preusmerila pot generiranja energije. Celica je začela producira acetat (`ac`), format (`for`) in etanol (`etoh`). Prej je producirala vodo in ogljikov dioksid.

In [87]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,0.3782,1,0.63%
glc__D_e,EX_glc__D_e,10,6,99.37%
h2o_e,EX_h2o_e,7.116,0,0.00%
nh4_e,EX_nh4_e,1.154,0,0.00%
pi_e,EX_pi_e,0.7786,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-8.504,2,33.11%
etoh_e,EX_etoh_e,-8.279,2,32.23%
for_e,EX_for_e,-17.8,1,34.66%
h_e,EX_h_e,-30.55,0,0.00%


Preberimo model Recon3D. Predtem smo ga prenesli s strani https://github.com/SBRG/Recon3D. Opis modela je na voljo v [članku](http://dx.doi.org/10.1038/nbt.4072).

In [31]:
model = cobra.io.read_sbml_model('models\\Recon3D.xml')

Poglejmo nekaj atributov modela

In [7]:
model.genes[:3]

[<Gene 0 at 0x2458db6b710>,
 <Gene 26_AT1 at 0x245a0236ad0>,
 <Gene 314_AT1 at 0x245a018ac10>]

In [8]:
model.metabolites[:3]

[<Metabolite 10fthf_c at 0x245a03511d0>,
 <Metabolite 10fthf_l at 0x245a0351150>,
 <Metabolite 10fthf_m at 0x245a03512d0>]

In [5]:
model.reactions[:3]

[<Reaction 24_25DHVITD3tm at 0x1eea06c6b90>,
 <Reaction 25HVITD3t at 0x1eef3d40a50>,
 <Reaction COAtl at 0x1eea06c7210>]

Povzetek modela lahko dobimo z metodo [summary](https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/summary/model_summary/index.html):

In [9]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
thmtp_c,DM_thmtp_c,1000,12,4.09%
12ppd__R_e,EX_12ppd__R_e,250.8,3,0.26%
5aop_e,EX_5aop_e,1000,5,1.70%
HC00250_e,EX_HC00250_e,1000,0,0.00%
HC00900_e,EX_HC00900_e,1000,4,1.36%
HC01361_e,EX_HC01361_e,1000,9,3.07%
Lkynr_e,EX_Lkynr_e,105.1,10,0.36%
acetone_e,EX_acetone_e,427.7,3,0.44%
alltn_e,EX_alltn_e,1000,4,1.36%
atp_e,EX_atp_e,1000,10,3.41%


In [32]:
rx = model.reactions.get_by_id("EX_glc__D_e")

In [23]:
rx.lower_bound=0
rx.upper_bound=0

In [24]:
rx

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x245ccc12950
Stoichiometry,glc__D_e --> D-Glucose -->
GPR,
Lower bound,0
Upper bound,0


In [33]:
s = model.optimize()

In [34]:
s["EX_glc__D_e"]

1000.0

In [37]:
rx

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x245dcce7a90
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [44]:
s

,fluxes,reduced_costs
24_25DHVITD3tm,0.0,0.000000e+00
25HVITD3t,0.0,0.000000e+00
COAtl,0.0,0.000000e+00
EX_5adtststerone_e,0.0,0.000000e+00
EX_5adtststerones_e,0.0,0.000000e+00
...,...,...
ACMPGLUTTRsc,0.0,2.088971e-18
FVSCOAhc,0.0,-9.638339e-02
MDZGLChr,0.0,0.000000e+00
TMACMPhr,0.0,2.389416e-17


### Shadow prices and reduced costs

[Source](https://edisciplinas.usp.br/mod/resource/view.php?id=2328722)


The sensitivity of an FBA solution is indicated by either shadow prices or reduced costs.  Shadow prices are the derivative of the objective function with respect to the exchange flux of a metabolite.  They indicate how much the addition of that metabolite will increase or decrease the objective.  Reduced costs are the derivatives of the objective function with respect to an internal reaction with 0 flux, indicating how much each particular reaction affects the objective.  In the COBRA Toolbox, shadow prices and reduced costs can be calculated by optimizeCbModel.  The vector of m shadow prices is FBAsolution.y and the vector of n reduced costs is FBAsoltuion.w.


**Shadow prices**

[Figure 1](https://www.researchgate.net/publication/256452494_Flux_Imbalance_Analysis_and_the_Sensitivity_of_Cellular_Growth_to_Changes_in_Metabolite_Pools/figures?lo=1). 

Shadow prices in FBA capture the sensitivity of growth to flux imbalances. Consider the FBA problem with one metabolite and two reactions, formulated as: , ; ; . The solid red line indicates the feasible solution space, and the red dot indicates the optimal solution. When the flux balance condition is relaxed and the outgoing flux from M is allowed to increase, the feasible space moves to the right (dashed blue line) and the optimal solution increases. Since the objective function increases as the right-hand-side of the flux balance constraint decreases, the metabolite has a negative shadow price. In general for intracellular metabolites, negative shadow prices correspond to growth-limiting metabolites.

In [46]:
s.shadow_prices

10fthf_c         2.228982e-18
10fthf_l         2.228982e-18
10fthf_m         2.228982e-18
11docrtsl_c      0.000000e+00
11docrtsl_m      0.000000e+00
                     ...     
caproic_e        0.000000e+00
1a25dhvitd2_c    0.000000e+00
1a25dhvitd2_e    0.000000e+00
protein_c        0.000000e+00
h_i             -4.669125e-18
Name: shadow_prices, Length: 5835, dtype: float64

In [35]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
thmtp_c,DM_thmtp_c,1000,12,4.09%
12ppd__R_e,EX_12ppd__R_e,250.8,3,0.26%
5aop_e,EX_5aop_e,1000,5,1.70%
HC00250_e,EX_HC00250_e,1000,0,0.00%
HC00900_e,EX_HC00900_e,1000,4,1.36%
HC01361_e,EX_HC01361_e,1000,9,3.07%
Lkynr_e,EX_Lkynr_e,105.1,10,0.36%
acetone_e,EX_acetone_e,427.7,3,0.44%
alltn_e,EX_alltn_e,1000,4,1.36%
atp_e,EX_atp_e,1000,10,3.41%


In [ ]:
rx.gene_reaction_rule

In [ ]:
for r in model.reactions[:100]:
    gpr = r.gene_reaction_rule
    if gpr:
        print(gpr)

In [ ]:
model.reactions.BIOMASS_maintenance.build_reaction_string()